In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score


import datasets
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained(
    "line-corporation/line-distilbert-base-japanese", trust_remote_code=True
)
model = AutoModel.from_pretrained("line-corporation/line-distilbert-base-japanese")

/home/imaseki/workspace/c103-text-classify/text_classify/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'DistilBertJapaneseTokenizer'.
Some weights of the model checkpoint at line-corporation/line-distilbert-base-japanese were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_t

In [2]:
train_df = pd.read_csv("../data/train.tsv", sep="\t")
valid_df = pd.read_csv("../data/valid.tsv", sep="\t")
test_df = pd.read_csv("../data/test.tsv", sep="\t")

len(train_df), len(valid_df), len(test_df)

(162, 54, 54)

In [3]:
# train_dfのpoemを入力にlabelを2値分類するfine-tuningを行う

from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

model = DistilBertForSequenceClassification.from_pretrained(
    "line-corporation/line-distilbert-base-japanese", num_labels=2
)
tokenizer = AutoTokenizer.from_pretrained(
    "line-corporation/line-distilbert-base-japanese", trust_remote_code=True
)


def tokenize(batch):
    return tokenizer(batch["poem"], padding=True, truncation=True)


train_dataset = datasets.Dataset.from_pandas(train_df)
valid_dataset = datasets.Dataset.from_pandas(valid_df)
test_dataset = datasets.Dataset.from_pandas(test_df)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
valid_dataset = valid_dataset.map(tokenize, batched=True, batch_size=len(valid_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
valid_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

training_args = TrainingArguments(
    output_dir="../data/output/line_distilbert/",
    overwrite_output_dir=True,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)
trainer.train()

Some weights of the model checkpoint at line-corporation/line-distilbert-base-japanese were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at line-corporation/line-distilbert-base-japanese and are newly initialized: ['classifi

Step,Training Loss


In [ ]:
preds = trainer.predict(test_dataset)
pred_labels = np.argmax(preds.predictions[0], axis=-1)

In [ ]:
# pred_labelsをvalidデータで評価する
accuracy_score(test_dataset["label"], pred_labels)

In [ ]:
# 単語ロジスティック回帰で誤っていた問題を解けていたか確認

test_df["pred"] = pred_labels
test_df[test_df["poem"].map(lambda x: "眼" in x)]